In [ ]:
# CEPEJ
import pandas as pd
import numpy as np
cepej = pd.read_excel('cepej_example.xlsx', index_col=0)
cepej = cepej.dropna()
cepej.head()

In [ ]:
y1 = 1000 - cepej['disposition time'].to_numpy()
y2 = cepej['clearance rate'].to_numpy()
Y = np.column_stack((y1, y2))
X = cepej[[
       "number_judges",
       "number_no-judges",
       "Information tools",
       "Tools of communication"
       ]].to_numpy()

In [ ]:
# Parameters
outputs = None
g_input = np.ones((X.shape[1]))
g_output = np.ones((Y.shape[1]))
constraint = "DRS"

In [ ]:
from Shapley import ShapleyModel
model = ShapleyModel(outputs = "multi-outputs", constraint = "DRS", g_inputs = np.ones((X.shape[1])), g_outputs =np.ones((Y.shape[1])))
DDF_results = model.dea_ddf(X,Y)
columns = ["DDF"]
df = pd.DataFrame(DDF_results, cepej.index, columns=columns)
df['Rank'] = df['DDF'].rank(ascending=True)
display(df)

In [ ]:
df = model.fit("shapley",X,Y)
columns = [
       "number_judges",
       "number_no-judges",
       "Information tools",
       "Tools of communication"
       ]
contrib_df = pd.DataFrame(df, index = cepej.index, columns = columns)
row_sums = contrib_df.sum(axis=1)
contrib_pourcentage = contrib_df.div(row_sums, axis=0) * 100
print("Absolute Contributions:", "\n")
display(contrib_df)
print("Contributions (%):", "\n")
display(contrib_pourcentage)

In [ ]:
df_ES = model.fit("ES",X,Y)
contrib_df_ES = pd.DataFrame(df_ES, index = cepej.index, columns = columns)

In [ ]:
# Plot
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(14, 8))
width = 0.35  
countries = cepej.index
for i, input in enumerate(columns):
    bar1 = ax.bar(np.arange(len(countries)) - width/2 + i*(width/4), contrib_df[input], width/4, label=f'{input} Shapley')
    bar2 = ax.bar(np.arange(len(countries)) + width/2 + i*(width/4), contrib_df_ES[input], width/4, label=f'{input} ES')
ax.set_xlabel('Countries')
ax.set_ylabel('Values')
ax.set_title('Comparison of Shapley and ES')
ax.set_xticks(np.arange(len(countries)))
ax.set_xticklabels(countries, rotation=90)
ax.legend()
plt.show()

In [ ]:
df_permutation = model.fit("permutation",X,Y)
contrib_df_permutation = pd.DataFrame(df_permutation, index = cepej.index, columns = columns)
display(contrib_df_permutation)